In [1]:
import pandas as pd
import psycopg2

In [2]:
con = psycopg2.connect(host='localhost', database='pop',
    user='postgres', password='root')
cursor = con.cursor()
print("Connected")

Connected


In [3]:
#empreendimentos
emp = pd.read_csv('./tab01/Empreendimento.csv',delimiter=",",encoding="latin-1")
emp = emp.loc[emp['PRODUTO']=='SOJA']
emp = emp.drop(['DATA_INICIO','DATA_FIM','UNIDADE_MEDIDA','UNIDADE_MEDIDA_PREVISAO','CONSORCIO','CEDULA_MAE','CD_TIPO_CULTURA'],axis=1)
emp.dropna(subset=['FINALIDADE'],inplace=True)
emp.dropna(subset=['ATIVIDADE'],inplace=True)
emp.dropna(subset=['MODALIDADE'],inplace=True)
emp.dropna(subset=['PRODUTO'],inplace=True)
emp.dropna(subset=['VARIEDADE'],inplace=True)
emp.dropna(subset=['CESTA'],inplace=True)
emp['ZONEAMENTO'] = emp['ZONEAMENTO'].fillna('Não zoneado')
emp.head(5)

,#CODIGO,FINALIDADE,ATIVIDADE,MODALIDADE,PRODUTO,VARIEDADE,CESTA,ZONEAMENTO
56,12016720000052,Custeio,Agrícola,LAVOURA,SOJA,NÃO SE APLICA,Irrigadas,Não zoneado
57,12016720000013,Custeio,Agrícola,LAVOURA,SOJA,NÃO SE APLICA,Safra de Verão (1ª Safra),Não zoneado
58,12016720000023,Custeio,Agrícola,LAVOURA,SOJA,NÃO SE APLICA,Safrinha (2ª Safra),Não zoneado
59,12016720000812,Custeio,Agrícola,LAVOURA,SOJA,NÃO SE APLICA,Safra de Verão (1ª Safra),Não zoneado
60,12016720000051,Custeio,Agrícola,LAVOURA,SOJA,NÃO SE APLICA,Irrigadas,Zoneado


In [11]:
#criando tipos
finalidade = [f"'{i}'" for i in emp['FINALIDADE'].unique()]
atividade = [f"'{i}'" for i in emp['ATIVIDADE'].unique()]
modalidade = [f"'{i}'" for i in emp['MODALIDADE'].unique()]
produto = [f"'{i}'" for i in emp['PRODUTO'].unique()]
variedade = [f"'{i}'" for i in emp['VARIEDADE'].unique()]
cesta = [f"'{i}'" for i in emp['CESTA'].unique()]
zoneamento = [f"'{i}'" for i in emp['ZONEAMENTO'].unique()]
file = open('./dll/types.sql','w',encoding='utf-8')
file.write(f'CREATE TYPE finalidade_type AS ENUM ({",".join(finalidade)});\n')
file.write(f'CREATE TYPE atividade_type AS ENUM ({",".join(atividade)});\n')
file.write(f'CREATE TYPE modalidade_type AS ENUM ({",".join(modalidade)});\n')
file.write(f'CREATE TYPE produto_type AS ENUM ({",".join(produto)});\n')
file.write(f'CREATE TYPE variedade_type AS ENUM ({",".join(variedade)});\n')
file.write(f'CREATE TYPE cesta_type AS ENUM ({",".join(cesta)});\n')
file.write(f'CREATE TYPE zoneamento_type AS ENUM ({",".join(zoneamento)});')
file.close()

In [19]:
#popular empreendimentos
file = open('./dll/data/empreendimentos.sql','w',encoding='utf-8')
for i in emp.index:
    sql = f'''INSERT INTO emp_empreendimento (
        emp_id,
        emp_finalidade,
        emp_atividade,
        emp_modalidade,
        emp_produto,
        emp_variedade,
        emp_cesta,
        emp_zoneamento
    ) VALUES (
        {str(emp['#CODIGO'][i])},
        '{emp['FINALIDADE'][i]}',
        '{emp['ATIVIDADE'][i]}',
        '{emp['MODALIDADE'][i]}',
        '{emp['PRODUTO'][i]}',
        '{emp['VARIEDADE'][i]}',
        '{emp['CESTA'][i]}',
        '{emp['ZONEAMENTO'][i]}'
    );'''
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [20]:
#ciclos
file = open('./dll/data/ciclo.sql','w',encoding='utf-8')
ciclos = pd.read_csv("./tab01/CicloCultivarProagro.csv",delimiter=";",encoding="latin-1")
for ciclo in ciclos.index:
    sql = "INSERT INTO ccl_ciclo (ccl_id,ccl_descricao) VALUES ("+str(ciclos['#CODIGO_CICLO'][ciclo])+",'"+ciclos['DESCRICAO_CICLO'][ciclo]+"');"
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [21]:
#cultivo
file = open('./dll/data/cultivo.sql','w',encoding='utf-8')
cultivos = pd.read_csv("./tab01/TipoCultivo.csv",delimiter=",",encoding="latin-1")
for cultivo in cultivos.index:
    sql = "INSERT INTO clt_cultivo (clt_id,clt_descricao) VALUES ("+str(cultivos['#CODIGO'][cultivo])+",'"+cultivos['DESCRICAO'][cultivo]+"');"
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [22]:
#grao e semente
file = open('./dll/data/grao_semente.sql','w',encoding='utf-8')
gsm = pd.read_csv("./tab01/GraoSemente.csv",delimiter=",",encoding="latin-1")
for i in gsm.index:
    sql = "INSERT INTO gsm_grao_semente (gsm_id,gsm_descricao) VALUES ("+str(gsm['#CODIGO'][i])+",'"+gsm['DESCRICAO'][i]+"');"
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [23]:
#irrigacao
file = open('./dll/data/irrigacao.sql','w',encoding='utf-8')
irrigacao = pd.read_csv("./tab01/TipoIrrigacao.csv",delimiter=",",encoding="latin-1")
for i in irrigacao.index:
    sql = "INSERT INTO irg_irrigacao (irg_id,irg_descricao) VALUES ("+str(irrigacao['#CODIGO'][i])+",'"+irrigacao['DESCRICAO'][i]+"');"
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [24]:
#solo
file = open('./dll/data/solo.sql','w',encoding='utf-8')
solo = pd.read_csv("./tab01/TipoSoloProagro.csv",delimiter=";",encoding="latin-1")
for i in solo.index:
    sql = "INSERT INTO sol_solo (sol_id,sol_descricao) VALUES ("+str(solo['#CODIGO_TIPO_SOLO'][i])+",'"+solo['DESCRICAO_TIPO_SOLO'][i]+"');"
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [4]:
#operacoes
op = pd.DataFrame(columns=['ID','EMP','UF','IRRIGACAO','CULTIVO','GRAO_SEMENTE'])
chunks = pd.read_csv('./tab02/SICOR_OPERACAO_BASICA_ESTADO_2018_ATUAL.csv',delimiter=";",iterator=True,chunksize=100000)
for chunk in chunks:
    data = chunk[chunk['CD_EMPREENDIMENTO'].isin(emp['#CODIGO'])]
    data['#REF_BACEN']
    new_line = pd.DataFrame({
            'ID':data['#REF_BACEN'].astype(str)+data['NU_ORDEM'].astype(str),
            'EMP':data['CD_EMPREENDIMENTO'],
            'UF':data['CD_ESTADO'].astype(str),
            'IRRIGACAO':data['CD_TIPO_IRRIGACAO'],
            'CULTIVO':data['CD_TIPO_CULTIVO'],
            'GRAO_SEMENTE':data['CD_TIPO_GRAO_SEMENTE']
        })
    op = pd.concat([op,new_line])
op.reset_index(drop=True, inplace=True)
op.head(5)

C:\Users\Pedro Motta\AppData\Local\Temp\ipykernel_1304\3087926892.py:4: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\Pedro Motta\AppData\Local\Temp\ipykernel_1304\3087926892.py:4: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\Pedro Motta\AppData\Local\Temp\ipykernel_1304\3087926892.py:4: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\Pedro Motta\AppData\Local\Temp\ipykernel_1304\3087926892.py:4: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\Pedro Motta\AppData\Local\Temp\ipykernel_1304\3087926892.py:4: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\Pedro Motta

,ID,EMP,UF,IRRIGACAO,CULTIVO,GRAO_SEMENTE
0,5104094661,12016720000011,PR,1,0,8
1,5120178411,12016720000011,PR,1,0,8
2,5073360761,12016720000012,MT,1,0,0
3,5144440901,12016720000011,RS,1,0,8
4,5087149171,12016720000011,RS,1,2,8


In [5]:
basic = pd.read_csv('./tab02/SICOR_COP_BASICO.csv',delimiter=';')
basic['ID'] = basic['#REF_BACEN'].astype(str)+basic['NU_ORDEM'].astype(str)
basic = basic[basic['ID'].isin(op['ID'])]
basic = basic.drop(['#REF_BACEN','NU_ORDEM','DT_COMUNICACAO','CD_STATUS','CD_EVENTO'],axis=1)
basic.head(5)

,DT_FIM_COLHEITA,DT_FIM_PLANTIO,DT_INICIO_COLHEITA,DT_INICIO_PLANTIO,CD_CICLO_CULTIVAR,CD_TIPO_SOLO,ID
0,03/20/2020 00:00:00,11/10/2019 00:00:00,03/16/2020 00:00:00,11/01/2019 00:00:00,3,3,5087781901
1,03/30/2020 00:00:00,11/18/2019 00:00:00,03/16/2020 00:00:00,11/15/2019 00:00:00,1,3,5086513751
3,05/08/2022 00:00:00,12/20/2021 00:00:00,05/07/2022 00:00:00,11/01/2021 00:00:00,3,3,5127358961
9,03/25/2022 00:00:00,11/20/2021 00:00:00,02/17/2022 00:00:00,10/15/2021 00:00:00,1,3,5122522651
13,04/15/2020 00:00:00,12/15/2019 00:00:00,03/26/2020 00:00:00,11/21/2019 00:00:00,1,2,5088670791


In [6]:
operacao = pd.merge(op,basic,on='ID', how='inner')
operacao['IRRIGACAO'] = operacao['IRRIGACAO'].astype(int)
operacao['CULTIVO'] = operacao['CULTIVO'].astype(int)
operacao['GRAO_SEMENTE'] = operacao['GRAO_SEMENTE'].astype(int)
operacao['DT_FIM_COLHEITA'] = pd.to_datetime(operacao['DT_FIM_COLHEITA'],format='%m/%d/%Y %H:%M:%S', errors='coerce')
operacao['DT_FIM_PLANTIO'] = pd.to_datetime(operacao['DT_FIM_PLANTIO'],format='%m/%d/%Y %H:%M:%S', errors='coerce')
operacao['DT_INICIO_COLHEITA'] = pd.to_datetime(operacao['DT_INICIO_COLHEITA'],format="%m/%d/%Y %H:%M:%S", errors='coerce')
operacao['DT_INICIO_PLANTIO'] = pd.to_datetime(operacao['DT_INICIO_PLANTIO'],format="%m/%d/%Y %H:%M:%S", errors='coerce')
operacao.head(5)

,ID,EMP,UF,IRRIGACAO,CULTIVO,GRAO_SEMENTE,DT_FIM_COLHEITA,DT_FIM_PLANTIO,DT_INICIO_COLHEITA,DT_INICIO_PLANTIO,CD_CICLO_CULTIVAR,CD_TIPO_SOLO
0,5087149171,12016720000011,RS,1,2,8,2020-03-18,2019-11-30,2020-03-17,2019-11-20,2,3
1,5126285181,12016720000011,PR,1,0,8,2022-02-15,2021-10-06,2022-01-30,2021-10-06,1,3
2,5124156661,12016720000011,PR,1,0,8,2022-02-10,2021-10-13,2022-02-01,2021-09-30,99,3
3,5123177041,12016720000011,PR,1,0,8,2022-02-20,2021-10-13,2022-02-20,2021-10-13,1,3
4,5123185982,12016720000011,RS,1,0,8,2022-04-14,2021-12-04,2022-04-11,2021-12-02,1,3


In [15]:
estados = pd.DataFrame({'UF':operacao['UF'].unique()})
estados = estados.sort_values(['UF'])
estados['INDEX'] = estados.index
estados.head(5)

,UF,INDEX
10,BA,10
8,GO,8
6,MA,6
9,MG,9
3,MS,3


In [46]:
file = open('./dll/data/estado.sql','w',encoding='utf-8')
for i in estados.index:
    sql = f'''INSERT INTO std_estado 
    (
        std_id,
        std_descricao
    ) VALUES (
        {i},
        '{estados['UF'][i]}'
    );'''
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [16]:
operacao = operacao.merge(estados,left_on="UF",right_on="UF",how="left")
operacao.head(5)

,ID,EMP,UF,IRRIGACAO,CULTIVO,GRAO_SEMENTE,DT_FIM_COLHEITA,DT_FIM_PLANTIO,DT_INICIO_COLHEITA,DT_INICIO_PLANTIO,CD_CICLO_CULTIVAR,CD_TIPO_SOLO,INDEX
0,5087149171,12016720000011,RS,1,2,8,2020-03-18,2019-11-30,2020-03-17,2019-11-20,2,3,0
1,5126285181,12016720000011,PR,1,0,8,2022-02-15,2021-10-06,2022-01-30,2021-10-06,1,3,1
2,5124156661,12016720000011,PR,1,0,8,2022-02-10,2021-10-13,2022-02-01,2021-09-30,99,3,1
3,5123177041,12016720000011,PR,1,0,8,2022-02-20,2021-10-13,2022-02-20,2021-10-13,1,3,1
4,5123185982,12016720000011,RS,1,0,8,2022-04-14,2021-12-04,2022-04-11,2021-12-02,1,3,0


In [17]:
mun = pd.read_csv('./tab03/ComplementoOperacaoBasica.csv',delimiter=',')
mun['ID'] = mun['REF_BACEN'].astype(str)+mun['NU_ORDEM'].astype(str)
mun.drop(['REF_BACEN','NU_ORDEM','REF_BACEN_EFETIVO','AGENCIA_IF'],axis=1,inplace=True)
mun = mun[mun['ID'].isin(operacao['ID'].unique())]
mun.shape

(99146, 2)

In [18]:
import requests
import time
mun['MUN'] = ''
mont = 0
for i in mun['CD_IBGE_MUNICIPIO'].unique():
    req = requests.get(f'https://servicodados.ibge.gov.br/api/v1/localidades/municipios/{str(i)}')
    if req.status_code == 200:
        mun.loc[mun['CD_IBGE_MUNICIPIO']==i,'MUN'] = req.json()['nome']
    time.sleep(0.3)
    mont += 1
    print(f'{mont}/{len(mun["CD_IBGE_MUNICIPIO"].unique())}',end="\r",flush=True)
mun.head(10)

,CD_IBGE_MUNICIPIO,ID,MUN
408,4317400.0,5123193151,Santiago
840,4317756.0,5086844532,Santo Antônio do Planalto
1053,4318424.0,5122645831,São João da Urtiga
1402,4311270.0,5089335401,Lagoa dos Três Cantos
1515,4315057.0,5129464471,Porto Mauá
1652,4307815.0,5126506431,Estrela Velha
1934,4306304.0,5129893321,David Canabarro
2099,4307500.0,5126836371,Espumoso
2189,4120358.0,5124455411,Pranchita
2215,4315503.0,5089052361,Restinga Sêca


In [19]:
mun['CD_IBGE_MUNICIPIO'] = mun['CD_IBGE_MUNICIPIO'].apply(lambda x:"{:.0f}".format(x))
mun.head(10)

,CD_IBGE_MUNICIPIO,ID,MUN
408,4317400,5123193151,Santiago
840,4317756,5086844532,Santo Antônio do Planalto
1053,4318424,5122645831,São João da Urtiga
1402,4311270,5089335401,Lagoa dos Três Cantos
1515,4315057,5129464471,Porto Mauá
1652,4307815,5126506431,Estrela Velha
1934,4306304,5129893321,David Canabarro
2099,4307500,5126836371,Espumoso
2189,4120358,5124455411,Pranchita
2215,4315503,5089052361,Restinga Sêca


In [20]:
operacao = operacao.merge(mun,left_on='ID',right_on='ID',how='left')
operacao = operacao.dropna()
operacao = operacao.drop_duplicates(subset='ID')
operacao.head(10)

,ID,EMP,UF,IRRIGACAO,CULTIVO,GRAO_SEMENTE,DT_FIM_COLHEITA,DT_FIM_PLANTIO,DT_INICIO_COLHEITA,DT_INICIO_PLANTIO,CD_CICLO_CULTIVAR,CD_TIPO_SOLO,INDEX,CD_IBGE_MUNICIPIO,MUN
0,5087149171,12016720000011,RS,1,2,8,2020-03-18,2019-11-30,2020-03-17,2019-11-20,2,3,0,4307500,Espumoso
1,5126285181,12016720000011,PR,1,0,8,2022-02-15,2021-10-06,2022-01-30,2021-10-06,1,3,1,4102000,Assis Chateaubriand
2,5124156661,12016720000011,PR,1,0,8,2022-02-10,2021-10-13,2022-02-01,2021-09-30,99,3,1,4113452,Lindoeste
3,5123177041,12016720000011,PR,1,0,8,2022-02-20,2021-10-13,2022-02-20,2021-10-13,1,3,1,4101002,Ampére
4,5123185982,12016720000011,RS,1,0,8,2022-04-14,2021-12-04,2022-04-11,2021-12-02,1,3,0,4317509,Santo Ângelo
5,5070825831,12016720000011,PR,1,2,8,2019-02-15,2018-10-20,2019-01-20,2018-09-20,3,3,1,4114609,Marechal Cândido Rondon
6,5123005931,12016720000011,RS,1,0,3,2022-04-25,2021-12-10,2022-03-10,2021-10-25,2,3,0,4309126,Gramado dos Loureiros
7,5127789491,12016720000011,PR,1,0,8,2022-01-30,2021-09-24,2022-01-01,2021-09-23,99,3,1,4127700,Toledo
8,5124781271,12016720000011,RS,1,0,8,2022-03-23,2021-11-15,2022-02-21,2021-11-14,1,3,0,4309506,Guarani das Missões
9,5124014911,12016720000011,RS,1,0,8,2022-04-30,2021-12-20,2022-04-02,2021-11-10,99,3,0,4309001,Giruá


In [21]:
mun = mun.drop('ID',axis=1)
mun = mun.drop_duplicates(subset='CD_IBGE_MUNICIPIO')
mun.shape

(1173, 2)

In [86]:
file = open('./dll/data/municipio.sql','w',encoding='utf-8')
for i in mun.index:
    sql = f'''INSERT INTO mun_municipio (
        mun_id,
        mun_descricao
    ) VALUES (
        {mun['CD_IBGE_MUNICIPIO'][i]},
        '{str(mun['MUN'][i]).replace("'",'')}'
    );'''
    file.write(sql+"\n")
    cursor.execute(sql)
con.commit()
file.close()

In [89]:
file = open('./dll/data/operacao.sql','w',encoding='utf-8')
for i in operacao.index:
    sql = f'''INSERT INTO opr_operacao (
        opr_id,
        opr_inicio_plantio,
        opr_fim_plantio,
        opr_inicio_colheita,
        opr_fim_colheita,
        opr_std,
        opr_mun,
        opr_sol,
        opr_irg,
        opr_clt,
        opr_gsm,
        opr_ccl,
        opr_emp
    ) VALUES (
        {str(operacao['ID'][i])},
        '{str(operacao['DT_INICIO_PLANTIO'][i])}',
        '{str(operacao['DT_FIM_PLANTIO'][i])}',
        '{str(operacao['DT_INICIO_COLHEITA'][i])}',
        '{str(operacao['DT_FIM_COLHEITA'][i])}',
        {str(operacao['INDEX'][i])},
        {str(operacao['CD_IBGE_MUNICIPIO'][i])},
        {str(operacao['CD_TIPO_SOLO'][i])},
        {str(operacao['IRRIGACAO'][i])},
        {str(operacao['CULTIVO'][i])},
        {str(operacao['GRAO_SEMENTE'][i])},
        {str(operacao['CD_CICLO_CULTIVAR'][i])},
        {str(operacao['EMP'][i])}
    );'''
    file.write(sql+'\n')
    cursor.execute(sql)
con.commit()
file.close()

In [90]:
#glebas
glebas = pd.DataFrame(columns=['OPR','LAT','LONG','ORDEM'])
gchunks = pd.read_csv('./tab03/Glebas.csv',delimiter=',',iterator=True,chunksize=100000)
for chunk in gchunks:
    id = chunk['REF_BACEN'].astype(str)+chunk['NU_ORDEM'].astype(str)
    data = chunk[id.isin(operacao['ID'])]
    new_line = pd.DataFrame({
            'OPR':data['REF_BACEN'].astype(str)+data['NU_ORDEM'].astype(str),
            'LAT':data['VL_LATITUDE'],
            'LONG':data['VL_LONGITUDE'],
            'ID':data['NU_IDENTIFICADOR'].astype(str)+data['NU_INDICE_GLEBA'].astype(str),
            'ORDEM':data['NU_INDICE_PONTO'],
        })
    glebas = pd.concat([glebas,new_line])
glebas.reset_index(drop=True, inplace=True)
glebas.head(5)

,OPR,LAT,LONG,ORDEM,ID
0,5073232731,"-24,478907","-53,648176",11,7250860
1,5087569411,"-28,749436","-52,635729",22,10
2,5126215741,"-29,717522","-52,557993",23,32
3,5093264711,"-28,689093","-54,034045",8,21
4,5129301361,"-31,186071","-52,63392",4,50784043


In [91]:
glebas['COORD'] = list(zip(glebas['LONG'], glebas['LAT']))
glebas = glebas.sort_values(['ID','ORDEM'])
glebas = glebas.groupby(['OPR','ID'])['COORD'].agg(list).reset_index()
glebas.head(5)

,OPR,ID,COORD
0,5061070671,3308850,"[(-51,930153, -28,33702), (-51,930211, -28,336..."
1,5061159921,5408390,"[(-53,687729, -30,155289), (-53,690907, -30,15..."
2,5061159921,5413962,"[(-53,451682, -30,1353), (-53,452995, -30,1314..."
3,5061159921,5413971,"[(-53,465268, -30,110952), (-53,457244, -30,10..."
4,5061159921,5413983,"[(-53,683972, -30,155284), (-53,685599, -30,15..."


In [92]:
def fix_coord(coord):
    if coord[0] != coord[-1]:
        coord.append(coord[0])
    return coord
glebas['COORD'] = glebas['COORD'].apply(fix_coord)

In [93]:
glebas['GEOM'] = glebas['COORD'].apply(lambda coord: "POLYGON(("+(",".join([f'{str(x).replace(",",".")} {str(y).replace(",",".")}' for x,y in coord]))+"))")
glebas.head(10)

,OPR,ID,COORD,GEOM
0,5061070671,3308850,"[(-51,930153, -28,33702), (-51,930211, -28,336...","POLYGON((-51.930153 -28.33702,-51.930211 -28.3..."
1,5061159921,5408390,"[(-53,687729, -30,155289), (-53,690907, -30,15...","POLYGON((-53.687729 -30.155289,-53.690907 -30...."
2,5061159921,5413962,"[(-53,451682, -30,1353), (-53,452995, -30,1314...","POLYGON((-53.451682 -30.1353,-53.452995 -30.13..."
3,5061159921,5413971,"[(-53,465268, -30,110952), (-53,457244, -30,10...","POLYGON((-53.465268 -30.110952,-53.457244 -30...."
4,5061159921,5413983,"[(-53,683972, -30,155284), (-53,685599, -30,15...","POLYGON((-53.683972 -30.155284,-53.685599 -30...."
5,5061209041,5632480,"[(-43,717555, -6,04591), (-43,710981, -6,04524...","POLYGON((-43.717555 -6.04591,-43.710981 -6.045..."
6,5061668251,5609150,"[(-53,836773, -30,199152), (-53,831311, -30,19...","POLYGON((-53.836773 -30.199152,-53.831311 -30...."
7,5062272321,1773710,"[(-53,28379, -27,11461), (-53,28418, -27,11398...","POLYGON((-53.28379 -27.11461,-53.28418 -27.113..."
8,5064354291,10,"[(-55,32933, -21,301001), (-55,331867, -21,298...","POLYGON((-55.32933 -21.301001,-55.331867 -21.2..."
9,5064802551,10,"[(-43,22298, -5,572107), (-43,22295, -5,572227...","POLYGON((-43.22298 -5.572107,-43.22295 -5.5722..."


In [94]:
file = open('./dll/data/glebas.sql','w',encoding='utf-8')
for i in glebas.index:
    sql = f'''INSERT INTO glb_gleba (
        glb_poligono,
        glb_opr
    ) VALUES (
        '{str(glebas['GEOM'][i])}',
        {str(glebas['OPR'][i])}
    );'''
    file.write(sql+'\n')
    cursor.execute(sql)
con.commit()
file.close()

In [7]:
pchunks = pd.read_csv('./tab03/Propriedades.csv',encoding='latin-1',iterator=True,chunksize=10000)
operacao = operacao.drop_duplicates(subset='ID')
propriedades = pd.DataFrame(columns=['ID','SNCR','NIRF','CAR'])
for chunk in pchunks:
    chunk['ID'] = chunk['REF_BACEN'].astype(str)+chunk['NU_ORDEM'].astype(str)
    data = chunk[chunk['ID'].isin(operacao['ID'])]
    new_line = pd.DataFrame({
            'ID':data['ID'],
            'DOC':data['CD_CNPJ_CPF'],
            'SNCR':data['CD_SNCR'],
            'NIRF':data['CD_NIRF'],
            'CAR':data['CD_CAR']
        })
    propriedades = pd.concat([propriedades,new_line])
propriedades.reset_index(drop=True, inplace=True)
propriedades.head(5)

C:\Users\Pedro Motta\AppData\Local\Temp\ipykernel_1304\1262968135.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  propriedades = pd.concat([propriedades,new_line])


,ID,SNCR,NIRF,CAR,DOC
0,5126749831,NaN,NaN,RS4321956265BFA111D054A919299518E228C4429,2.564897e+10
1,5068697201,NaN,NaN,NaN,8.392449e+08
2,5123704661,NaN,NaN,RS4304655B7B9CEB74FE5450F93518F5B0EBE9CB4,2.245012e+10
3,5087644321,NaN,NaN,RS4315354E232A1AB1CE74BE9ABA7592DE215F8EC,2.458742e+10
4,5086705341,NaN,NaN,RS430150391CCC245D2C343589688606F02DE5D0F,3.977991e+10


In [8]:
propriedades = propriedades.dropna(subset=['SNCR','NIRF','CAR'],how='all')
propriedades.shape


(168273, 5)

In [ ]:
propriedades['DOC'] = propriedades['DOC'].astype(str)

In [31]:
propriedades.head(10)

,ID,SNCR,NIRF,CAR
0,5126749831,NaN,NaN,RS4321956265BFA111D054A919299518E228C4429
2,5123704661,NaN,NaN,RS4304655B7B9CEB74FE5450F93518F5B0EBE9CB4
3,5087644321,NaN,NaN,RS4315354E232A1AB1CE74BE9ABA7592DE215F8EC
4,5086705341,NaN,NaN,RS430150391CCC245D2C343589688606F02DE5D0F
5,5090185212,NaN,NaN,RS430630445900CDFDEDC41198E4C0CAD2A622AB8
6,5124879311,NaN,NaN,PR41280050E881AEBD715492DB8FEEB42DB0877CB
7,5123509571,NaN,NaN,RS4315107008005E9BA394D6A8154A2BFE8DA1D2A
8,5123272011,NaN,NaN,RS4301503C60DB3EC009341C7ADE51D6111B60DFD
9,5089417191,NaN,NaN,RS4314605A460B72288E041D3A1E01D57FFF601CC
10,5124279742,NaN,NaN,RS4300208FB50A866573D4D39A80AA0DE68ADCEE9


In [33]:
file = open('./dll/data/propriedades.sql','w',encoding='utf-8')
for i in propriedades.index:
    sql = f'''INSERT INTO prp_propriedade (
        prp_sncr,
        prp_doc,
        prp_nirf,
        prp_car,
        prp_opr
    ) VALUES (
        {'null' if pd.isna(propriedades['SNCR'][i]) else "'"+str(propriedades['SNCR'][i])+"'"},
        '{propriedades['DOC'][i].astype(str)}',
        {'null' if pd.isna(propriedades['NIRF'][i]) else "'"+str(propriedades['NIRF'][i])+"'"},
        {'null' if pd.isna(propriedades['CAR'][i]) else "'"+str(propriedades['CAR'][i])+"'"},
        {propriedades['ID'][i]}
    );'''
    file.write(sql+'\n')
    cursor.execute(sql)
con.commit()
file.close()

In [32]:
con.close()
file.close()